# Vision support
https://platform.claude.com/docs/en/build-with-claude/vision

In [ ]:
import base64
from os import environ
from typing import List, Dict, Iterable, Union, Any

# Load env variables and create client
from anthropic import AnthropicVertex
from anthropic.types import (
    TextBlockParam,
    ImageBlockParam,
    Message,
    ToolParam,
    Base64ImageSourceParam,
    Base64PDFSourceParam,
    DocumentBlockParam,
    ContentBlockParam,
)

region = environ.get("CLOUD_ML_REGION", "")
project_id = environ.get("ANTHROPIC_VERTEX_PROJECT_ID", "")
client = AnthropicVertex(region=region, project_id=project_id)
model = "claude-sonnet-4-5@20250929"

In [ ]:
# Helper functions
def add_user_message(messages: List[dict], msg: Any):
    """Adds a user message to the messages list."""
    user_message = {
        "role": "user",
        "content": msg.content if isinstance(msg, Message) else msg,
    }
    messages.append(user_message)


def add_assistant_message(messages: List[dict], msg: Union[Message, dict, str]):
    """Adds an assistant message to the messages list."""
    assistant_message = {
        "role": "assistant",
        "content": msg.content if isinstance(msg, Message) else msg,
    }
    messages.append(assistant_message)


def chat(
    messages: List[dict],
    system: str = None,
    temperature: float = 1.0,
    stop_sequences: Iterable[str] = [],
    tools: Iterable[ToolParam] = None,
    tool_choice: Dict = None,
    thinking: bool = False,
    thinking_budget: int = 1024,
):
    """Sends a chat request to the Claude model with the given messages."""
    params = {
        "model": model,
        "max_tokens": 4000,
        "messages": messages,
        "temperature": temperature,
        "stop_sequences": stop_sequences,
    }

    if system is not None:
        params["system"] = system

    if tools is not None:
        params["tools"] = tools

    if tool_choice is not None:
        params["tool_choice"] = tool_choice

    if thinking:
        params["thinking"] = {
            "type": "enabled",
            "budget_tokens": thinking_budget,
        }

    message = client.messages.create(**params)
    return message


def text_from_message(message: Message) -> Iterable[str]:
    """Extracts and concatenates all text blocks from a Message object."""
    return "\n".join([block.text for block in message.content if block.type == "text"])

In [ ]:
prompt = """
Analyze the attached satellite image of a property with these specific steps:

1. Residence identification: Locate the primary residence on the property by looking for:
   - The largest roofed structure
   - Typical residential features (driveway connection, regular geometry)
   - Distinction from other structures (garages, sheds, pools)

2. Tree overhang analysis: Examine all trees near the primary residence:
   - Identify any trees whose canopy extends directly over any portion of the roof
   - Estimate the percentage of roof covered by overhanging branches (0-25%, 25-50%, 50-75%, 75%+)
   - Note particularly dense areas of overhang

3. Fire risk assessment: For any overhanging trees, evaluate:
   - Potential wildfire vulnerability (ember catch points, continuous fuel paths to structure)
   - Proximity to chimneys, vents, or other roof openings if visible
   - Areas where branches create a "bridge" between wildland vegetation and the structure

4. Defensible space identification: Assess the property's overall vegetative structure:
   - Identify if trees connect to form a continuous canopy over or near the home
   - Note any obvious fuel ladders (vegetation that can carry fire from ground to tree to roof)

5. Fire risk rating: Based on your analysis, assign a Fire Risk Rating from 1-4:
   - Rating 1 (Low Risk): No tree branches overhanging the roof, good defensible space around
   - Rating 2 (Moderate Risk): Minimal overhang (<25% of roof), some separation between tree canopies
   - Rating 3 (High Risk): Significant overhang (25-50% of roof), connected tree canopies, multiple points of vulnerability
   - Rating 4 (Severe Risk): Extensive overhang (>50% of roof), dense vegetation against structure

For each item above (1-5), write one sentence summarizing your findings, with your final response being the numerical rating.
 """

In [ ]:
# Optional: Add strict runtime validation using Pydantic TypeAdapter
from pydantic import TypeAdapter, ValidationError
from typing import List

ContentBlockValidator = TypeAdapter(ContentBlockParam)


def validate_content_blocks(input_blocks: List[Any]) -> List[ContentBlockParam]:
    """Validates a list of content blocks against the ContentBlockParam schema."""
    validated_blocks = []
    for block in input_blocks:
        try:
            validated_block = ContentBlockValidator.validate_python(block, by_name=True)
            validated_blocks.append(validated_block)
        except ValidationError as e:
            print(f"Content block validation error: {e}")
            raise
    return validated_blocks

In [ ]:
with open("006-images/prop7.png", "rb") as f:
    image_data = base64.standard_b64encode(f.read()).decode("utf-8")

# Create properly typed and validated blocks
image_source: Base64ImageSourceParam = {
    "type": "base64",
    "media_type": "image/png",
    "data": image_data,
}

image_block: ImageBlockParam = {
    "type": "image",
    "source": image_source,
}

text_block: TextBlockParam = {
    "type": "text",
    "text": prompt,
}

messages: List[dict] = []

# Create the messages list with validated content
blocks: List[ContentBlockParam] = validate_content_blocks([image_block, text_block])
# Serialize Pydantic models back to dicts for API compatibility
serialized_blocks = [
    block.model_dump(mode="python") if hasattr(block, "model_dump") else block
    for block in blocks
]
add_user_message(messages, serialized_blocks)
rest = chat(messages)
text_from_message(rest)

# PDF Support

In [ ]:
messages: List[dict] = []

with open("006-images/earth.pdf", "rb") as f:
    file_data = base64.standard_b64encode(f.read()).decode("utf-8")

# Create properly typed and validated blocks
file_source: Base64PDFSourceParam = {
    "type": "base64",
    "media_type": "application/pdf",
    "data": file_data,
}

file_block: DocumentBlockParam = {
    "type": "document",
    "source": file_source,
}

text_block: TextBlockParam = {
    "type": "text",
    "text": "Summarise document in a single sentence.",
}

# Create the messages list with validated content
blocks: List[ContentBlockParam] = validate_content_blocks([file_block, text_block])
# Serialize Pydantic models back to dicts for API compatibility
serialized_blocks = [
    block.model_dump(mode="python") if hasattr(block, "model_dump") else block
    for block in blocks
]
add_user_message(messages, serialized_blocks)
rest = chat(messages)
text_from_message(rest)

## Citations
The citations feature lets you show users exactly where Claude found its information, building trust and transparency into your AI applications.

In [ ]:
messages: List[dict] = []
from json import dumps

with open("006-images/earth.pdf", "rb") as f:
    file_data = base64.standard_b64encode(f.read()).decode("utf-8")

# Create properly typed and validated blocks
file_source: Base64PDFSourceParam = {
    "type": "base64",
    "media_type": "application/pdf",
    "data": file_data,
}

file_block: DocumentBlockParam = {
    "type": "document",
    "source": file_source,
    "citations": {"enabled": True},
    "title": "earth.pdf",
}

text_block: TextBlockParam = {
    "type": "text",
    "text": "How was Earth's atmosphere formed and oceans formed?",
}

# Create the messages list with validated content
blocks: List[ContentBlockParam] = validate_content_blocks([file_block, text_block])
# Serialize Pydantic models back to dicts for API compatibility
serialized_blocks = [
    block.model_dump(mode="python") if hasattr(block, "model_dump") else block
    for block in blocks
]
add_user_message(messages, serialized_blocks)
rest = chat(messages)

In [ ]:
for block in rest.content:
    if block.type == "text":
        print("Answer:")
        print(block.text)
        if hasattr(block, "citations") and block.citations:
            print("\nCitations:")
            for citation in block.citations:
                print(dumps(citation.model_dump(mode="python"), indent=2))